# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from neural_data_analysis.design_kits.design_by_segment import create_design_df, predictor_utils, other_feats
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event, pn_glm_utils
from neural_data_analysis.neural_analysis_tools.glm_tools.tpg import glm_bases, glm_plotting, glm_plotting2, glm_fit
from neural_data_analysis.neural_analysis_tools.decoding_tools.general_decoding import standard_decoding

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_decoding_tools import glm_decoding_llr, glm_decoding
from planning_analysis.show_planning.cur_vs_nxt_ff import cvn_from_ref_class
from planning_analysis.plan_factors import build_factor_comp


import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
from numpy import pi

from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0222"

In [ ]:
cvn = cvn_from_ref_class.CurVsNxtFfFromRefClass(raw_data_folder_path=raw_data_folder_path)
# Quick method - tries to retrieve first, creates if needed
cvn.make_heading_info_df_without_long_process(
    test_or_control='test',  # or 'control'
    ref_point_mode='distance',  # or 'time after cur ff visible'
    ref_point_value=-100,  # or 0.0 for time mode
    heading_info_df_exists_ok=True,  # Set to False to force recreation
    stops_near_ff_df_exists_ok=True,
    save_data=True
)

# Access the result
heading_info_df = cvn.heading_info_df
heading_df = heading_info_df[['cur_ff_index', 'diff_in_abs_angle_to_nxt_ff']].copy()
heading_df = heading_df.sort_values(by='diff_in_abs_angle_to_nxt_ff', ascending=False).reset_index(drop=True)

# based on same side

In [ ]:
from data_wrangling import combine_info_utils, specific_utils

# Get all sessions for a specific monkey
monkey_name = "monkey_Bruno"  # or "monkey_Schro"
sessions_df = combine_info_utils.make_sessions_df_for_one_monkey(
    raw_data_dir_name='all_monkey_data/raw_monkey_data',
    monkey_name=monkey_name
)

# Iterate through each session
for index, row in sessions_df.iterrows():
    if row['finished']:
        continue  # Skip already processed sessions
    
    # Construct the raw_data_folder_path
    raw_data_folder_path = f"all_monkey_data/raw_monkey_data/{row['monkey_name']}/{row['data_name']}"
    
    print(f"Processing: {raw_data_folder_path}")
    

    pn = glm_decoding.init_decoding_data(raw_data_folder_path)

    heading_info_df, heading_df = pn_glm_utils.get_test_heading_df(raw_data_folder_path)

    build_factor_comp.add_dir_from_cur_ff_same_side(heading_info_df)
    heading_info_df['dir_from_cur_ff_same_side'].mean()


    for same_side in [True, False]:
        print('-'*100)
        print('-'*100)
        if same_side:
            str = "=========Same Side========="
        else:
            str = "=========Opposite Side========="
            
        rebinned_x_var, rebinned_y_var = pn_glm_utils.select_ff_subset_by_dir_from_cur_ff_same_side(heading_info_df, pn.rebinned_x_var, pn.rebinned_y_var,
                                                                                                    same_side=same_side)
        
        rebinned_x_var = pn_glm_utils.drop_constant_columns(rebinned_x_var)
        data = rebinned_y_var.copy()


        df_X, df_Y = glm_decoding.get_data_for_decoding_vis(rebinned_x_var, rebinned_y_var, pn.bin_width)

        exposure = np.ones(len(df_Y)) * pn.bin_width
        offset_log = np.log(exposure)

        report = stop_glm_fit.glm_mini_report(
            df_X=df_X, df_Y=df_Y, offset_log=offset_log,
            cov_type='HC1', 
            fast_mle=True,
            do_inference=False, 
            make_plots=False,
            show_plots=True,
        )
        
        #cols_to_decode = ['nxt_vis', 'random_0_or_1', 'cur_vis']
        cols_to_decode = ['nxt_vis']
        groups = np.array(data['new_segment'])

        # # Decoding from fit
        # print(f"{str}")
        # glm_decoding.glm_decoding_from_fit(cols_to_decode, df_X, df_Y, offset_log, report)

        # CV
        print(f"{str}")
        glm_decoding.glm_decoding_cv(cols_to_decode, df_X, df_Y, groups, offset_log)

        # # permutations
        # print(f"{str}")
        # glm_decoding.glm_decoding_permutation_test(cols_to_decode, df_X, df_Y,
        #                         groups, offset_log, report, print_progress=False)

# top vs bottom (diff_in_abs_angle_to_nxt_ff)

In [ ]:
from data_wrangling import combine_info_utils, specific_utils

# Get all sessions for a specific monkey
monkey_name = "monkey_Bruno"  # or "monkey_Schro"
sessions_df = combine_info_utils.make_sessions_df_for_one_monkey(
    raw_data_dir_name='all_monkey_data/raw_monkey_data',
    monkey_name=monkey_name
)

# Iterate through each session
for index, row in sessions_df.iterrows():
    if row['finished']:
        continue  # Skip already processed sessions
    
    # Construct the raw_data_folder_path
    raw_data_folder_path = f"all_monkey_data/raw_monkey_data/{row['monkey_name']}/{row['data_name']}"
    
    print(f"Processing: {raw_data_folder_path}")
    

    pn = glm_decoding.init_decoding_data(raw_data_folder_path)

    heading_info_df, heading_df = pn_glm_utils.get_test_heading_df(raw_data_folder_path)

    heading_info_df['dir_from_cur_ff_same_side'].mean()


    for top in [True, False]:
        print('-'*100)
        print('-'*100)
        if top:
            str = "=========TOP TOP TOP TOP TOP========="
        else:
            str = "=========BOTTOM BOTTOM BOTTOM BOTTOM BOTTOM========="
        rebinned_x_var, rebinned_y_var = pn_glm_utils.select_ff_subset(heading_df, pn.rebinned_x_var, pn.rebinned_y_var, 
                                                                    top=False, pct=0.5)

        rebinned_x_var = pn_glm_utils.drop_constant_columns(rebinned_x_var)
        data = rebinned_y_var.copy()


        df_X, df_Y = glm_decoding.get_data_for_decoding_vis(rebinned_x_var, rebinned_y_var, pn.bin_width)

        exposure = np.ones(len(df_Y)) * pn.bin_width
        offset_log = np.log(exposure)

        report = stop_glm_fit.glm_mini_report(
            df_X=df_X, df_Y=df_Y, offset_log=offset_log,
            cov_type='HC1', 
            fast_mle=True,
            do_inference=False, 
            make_plots=False,
            show_plots=True,
        )
        
        #cols_to_decode = ['nxt_vis', 'random_0_or_1', 'cur_vis']
        cols_to_decode = ['nxt_vis']
        groups = np.array(data['new_segment'])

        # Decoding from fit
        print(f"{str}")
        glm_decoding.glm_decoding_from_fit(cols_to_decode, df_X, df_Y, offset_log, report)

        # CV
        print(f"{str}")
        glm_decoding.glm_decoding_cv(cols_to_decode, df_X, df_Y, groups, offset_log)

        # permutations
        print(f"{str}")
        glm_decoding.glm_decoding_permutation_test(cols_to_decode, df_X, df_Y,
                                groups, offset_log, report, print_progress=False)


# debug

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0222"
pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=True)

In [ ]:
pn.rebin_data_in_new_segments(cur_or_nxt='cur', first_or_last='first', time_limit_to_count_sighting=2,
                                pre_event_window=0, post_event_window=1.5, rebinned_max_x_lag_number=2)

In [ ]:
pn.rebinned_y_var = pn_utils.rebin_segment_data(
    pn.planning_data_by_point, pn.new_seg_info, bin_width=pn.bin_width)

In [ ]:
pn.rebinned_y_var[['time_since_target_last_seen']].info()

In [ ]:
pn.rebinned_y_var['time_since_target_last_seen'].describe()

In [ ]:
pn.dec.behav_data_by_point

In [ ]:
pn.dec.get_basic_data()
pn.dec._make_or_retrieve_target_df(exists_ok=False)
pn.dec.make_or_retrieve_target_cluster_df()
pn.dec.target_df

In [ ]:
pn.rebinned_y_var.loc[pn.rebinned_y_var['time_since_target_last_seen'].isna(), ['cur_ff_index', 'cur_in_memory', 'time']]